# Text Generation with LSTM Neural Network

This project demonstrates how to generate text using an LSTM (Long Short-Term Memory) neural network. The dataset used is the text from a public domain book, and the model is trained to predict the next word in a sequence given a context of previous words.

In [1]:
import random
import pickle
import re

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
with open('1661-0.txt', 'r', encoding='utf-8') as file:
    text = file.read()

clean_text = re.sub(r'[^a-zA-Z\s]', '', text).lower()[:60000]

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(clean_text)

In [4]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [5]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [6]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [7]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])


C:\Users\Anurag Sunil\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
history = model.fit(X, y, batch_size=128, epochs=30, shuffle=True,validation_data=(X_val, y_val))
model.save("mymodel.keras")
model = load_model("mymodel.keras")


Epoch 1/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.0392 - loss: 6.7056 - val_accuracy: 0.0605 - val_loss: 6.1033
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.0601 - loss: 6.1380 - val_accuracy: 0.0767 - val_loss: 5.9083
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.0650 - loss: 5.9570 - val_accuracy: 0.0806 - val_loss: 5.7686
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.0735 - loss: 5.8003 - val_accuracy: 0.0854 - val_loss: 5.5714
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.0836 - loss: 5.5847 - val_accuracy: 0.0923 - val_loss: 5.2894
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.0969 - loss: 5.3927 - val_accuracy: 0.1159 - val_loss: 5.1065
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.1140 - loss: 5.1781 - val_accuracy: 0.1346 - val_loss: 4.8743
Epoch 8/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.1375 - loss: 4.9163 - val_accuracy: 0.1516 - v

In [16]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [23]:
possible = predict_next_word("He will have to do that thing for it is", 5)
print([unique_tokens[idx] for idx in possible])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
['always', 'spaulding', 'present', 'well', 'certainly']


In [24]:
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current + n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [25]:
possible = predict_next_word("He will have to do that thing for it is, 5)

SyntaxError: unterminated string literal (detected at line 1) (594570740.py, line 1)

In [26]:
print(generate_text("He will have to do that thing for it is", 100, 5))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━




This project demonstrates how to generate text using an LSTM (Long Short-Term Memory) neural network. The dataset used is the text from a public domain book, and the model is trained to predict the next word in a sequence given a context of previous words.

## Project Structure

- `1661-0.txt`: The text file containing the training data.
- `mymodel.keras`: The saved Keras model after training.

## Dependencies

- Python 3.x
- TensorFlow
- NumPy
- NLTK
- scikit-learn

Install the required libraries using:


## Data Preprocessing

The text data is preprocessed by:
- Removing non-alphabetic characters and converting to lowercase.
- Tokenizing the text into words.
- Creating a mapping of unique tokens to indices.
- Generating input-output pairs for the model where input is a sequence of 10 words and output is the next word.

## Model Architecture

The model used is a Sequential LSTM neural network with the following architecture:
- Two LSTM layers with 128 units each.
- Dense layer with the number of units equal to the number of unique tokens.
- Softmax activation for the output layer.

## Training

The model is trained using the categorical crossentropy loss function and RMSprop optimizer. The data is split into training and validation sets to evaluate validation accuracy.

## Text Generation

The model can generate text given a seed input. It predicts the next word based on the previous 10 words and appends it to the input sequence.


## Acknowledgements

This project is inspired by various tutorials and examples of text generation using LSTM networks. Special thanks to the authors of these resources.
